# Libraries

In [2]:
# generic imports
import pandas as pd
import numpy as np
import datetime

# data augmentation imports
from be_great import GReaT # https://github.com/kathrinse/be_great

## Load the Dataset

In [3]:
# Import DNN-EdgeIIoT-dataset.csv file from Kaggle (https://www.kaggle.com/datasets/mohamedamineferrag/edgeiiotset-cyber-security-dataset-of-iot-iiot)
df = pd.read_csv('../data/DNN-EdgeIIoT-dataset.csv')

/tmp/ipykernel_907213/1440379499.py:2: DtypeWarning: Columns (2,3,6,11,13,14,15,16,17,31,32,34,39,45,51,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/DNN-EdgeIIoT-dataset.csv')


#### AQUI DEVEM ESTAR AS TRANSFORMAÇÔES QUE O DATASET SOFREU ATÉ EdgeIIot_train_100k.csv

In [ ]:
# mbtcp.unit_id e mbtcp.trans_id devem ser removidos pois são identificadores

In [5]:
# Load data relative to the training set, with Attack_type "Normal" with 100k rows (SHORCUT)
df_train = pd.read_csv("../data/EdgeIIot_train_100k.csv", low_memory=False)

In [6]:
df_train.head()

,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,icmp.unused,http.content_length,http.request.method,http.referer,http.request.version,http.response,...,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,Attack_label,Attack_type
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DDoS_UDP
1,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Password
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DDoS_UDP
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DDoS_UDP
4,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Password


In [9]:
# list all columns
df_train.columns

Index(['arp.opcode', 'arp.hw.size', 'icmp.checksum', 'icmp.seq_le',
       'icmp.unused', 'http.content_length', 'http.request.method',
       'http.referer', 'http.request.version', 'http.response',
       'http.tls_port', 'tcp.ack', 'tcp.ack_raw', 'tcp.checksum',
       'tcp.connection.fin', 'tcp.connection.rst', 'tcp.connection.syn',
       'tcp.connection.synack', 'tcp.flags', 'tcp.flags.ack', 'tcp.len',
       'tcp.seq', 'udp.stream', 'udp.time_delta', 'dns.qry.name',
       'dns.qry.name.len', 'dns.qry.qu', 'dns.qry.type', 'dns.retransmission',
       'dns.retransmit_request', 'dns.retransmit_request_in',
       'mqtt.conack.flags', 'mqtt.conflag.cleansess', 'mqtt.conflags',
       'mqtt.hdrflags', 'mqtt.len', 'mqtt.msg_decoded_as', 'mqtt.msgtype',
       'mqtt.proto_len', 'mqtt.protoname', 'mqtt.topic', 'mqtt.topic_len',
       'mqtt.ver', 'mbtcp.len', 'mbtcp.trans_id', 'mbtcp.unit_id',
       'Attack_label', 'Attack_type'],
      dtype='object')

## Data Augmentation Model Training

In [8]:
# instantiate the GReaT model with the distilgpt2 language model
model = GReaT(llm="distilgpt2",              # Name of the large language model used (see HuggingFace for more options)
              epochs=10,                     # Number of epochs to train
              save_steps=2000,               # Save model weights every x steps
              logging_steps=500,             # Log the loss and learning rate every x steps
              experiment_dir="checkpoints/GReat", # Name of the directory where all intermediate steps are saved
              batch_size=16,                 # Batch Size
              #lr_scheduler_type="constant", # Specify the learning rate scheduler 
              #learning_rate=5e-5            # Set the inital learning rate
             )


# fit the model on the training data
model.fit(df_train)

## Save the model

In [ ]:
model.save("checkpoints/GReat/trained_model")

## Data Generation

In [ ]:
def sample_by_category(model, category_dict, columns):
    # create empty dataframe with columns 
    sampled_df = pd.DataFrame(columns=columns)
    for category, n_samples in category_dict.items():
        start_time = time.time()
        while len(sampled_df[sampled_df['Attack_type'] == category]) < n_samples:
            
            samples = model.sample(n_samples=20000)
            # select only the samples with the desired category
            category_samples = samples[samples['Attack_type'] == category]
            # discard the excess samples if necessary
            if len(category_samples) > n_samples:
                category_samples = category_samples.sample(n=n_samples)
            # add the selected samples to the sampled_df dataframe
            sampled_df = pd.concat([sampled_df, category_samples], ignore_index=True)
            
            elapsed_time = time.time() - start_time
            if elapsed_time > 5:
                clear_output(wait=True)
                print(f"{len(sampled_df[sampled_df['Attack_type'] == category])}/{n_samples}", end="\r")
                start_time = time.time()
    return sampled_df

In [ ]:
samples = model.sample(n_samples, k=50, start_col="target", start_col_dist={"0": 0.5, "1": 0.5, "2": 0})